### BallTree와 Haversine

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

In [2]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '/data/ephemeral/home/train.csv'
test_path  = '/data/ephemeral/home/test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)

address_path = '/data/ephemeral/home/address_data.csv'
dt_addr = pd.read_csv(address_path)

bus_path = '/data/ephemeral/home/bus_feature.csv'
subway_path = '/data/ephemeral/home/subway_feature.csv'
dt_bus = pd.read_csv(bus_path)
dt_subway = pd.read_csv(subway_path)

/tmp/ipykernel_523538/1278040586.py:4: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  dt = pd.read_csv(train_path)


In [3]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
dt['is_test'] = 0
dt_test['is_test'] = 1
concat = pd.concat([dt, dt_test])     # 하나의 데이터로 만들어줍니다.

In [4]:
dt_addr_selected = dt_addr.drop(axis=1, columns=['시군구', '번지', '도로명'])
display(dt_addr_selected.head(2))
display(dt_addr_selected.tail(2))

,좌표X,좌표Y,주소
0,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3
1,127.05721,37.476763,서울특별시 강남구 개포동 언주로 3


,좌표X,좌표Y,주소
1128092,127.063356,37.501358,서울특별시 중랑구 신내동 신내역로1길 85
1128093,127.063356,37.501358,서울특별시 중랑구 신내동 신내역로1길 85


In [5]:
display(concat.index)
display(dt_addr_selected.index)
dt_addr_selected.index = concat.index
display(concat.index)
display(dt_addr_selected.index)

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            9262, 9263, 9264, 9265, 9266, 9267, 9268, 9269, 9270, 9271],
           dtype='int64', length=1128094)

RangeIndex(start=0, stop=1128094, step=1)

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            9262, 9263, 9264, 9265, 9266, 9267, 9268, 9269, 9270, 9271],
           dtype='int64', length=1128094)

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            9262, 9263, 9264, 9265, 9266, 9267, 9268, 9269, 9270, 9271],
           dtype='int64', length=1128094)

In [6]:
#2개 데이터 프레임값 비교
display(concat[['시군구','번지','본번','도로명','좌표X','좌표Y']][975:976])
display(dt_addr_selected[975:976])

# 컬럼의 비어있는 값을 dt_xy_selected 컬럼 '좌표X','좌표Y' 값으로 채우기
concat['좌표X'] = concat['좌표X'].combine_first(dt_addr_selected['좌표X'])
concat['좌표Y'] = concat['좌표Y'].combine_first(dt_addr_selected['좌표Y'])

,시군구,번지,본번,도로명,좌표X,좌표Y
975,서울특별시 강남구 개포동,189,189.0,삼성로 14,NaN,NaN


,좌표X,좌표Y,주소
975,127.063356,37.501358,서울특별시 강남구 개포동 삼성로 14


In [7]:
display(concat[['시군구','번지','본번','도로명','좌표X','좌표Y']][970:976])
display(dt_addr_selected[970:976])

,시군구,번지,본번,도로명,좌표X,좌표Y
970,서울특별시 강남구 대치동,316,316.0,삼성로 212,127.065070,37.497741
971,서울특별시 강남구 대치동,316,316.0,삼성로 212,127.065070,37.497741
972,서울특별시 강남구 대치동,316,316.0,삼성로 212,127.065070,37.497741
973,서울특별시 강남구 대치동,316,316.0,삼성로 212,127.065070,37.497741
974,서울특별시 강남구 대치동,316,316.0,삼성로 212,127.065070,37.497741
975,서울특별시 강남구 개포동,189,189.0,삼성로 14,127.063356,37.501358


,좌표X,좌표Y,주소
970,127.065070,37.497741,서울특별시 강남구 대치동 삼성로 212
971,127.065070,37.497741,서울특별시 강남구 대치동 삼성로 212
972,127.065070,37.497741,서울특별시 강남구 대치동 삼성로 212
973,127.065070,37.497741,서울특별시 강남구 대치동 삼성로 212
974,127.065070,37.497741,서울특별시 강남구 대치동 삼성로 212
975,127.063356,37.501358,서울특별시 강남구 개포동 삼성로 14


In [13]:
# 지하철/버스 데이터프레임의 좌표 배열 생성
sub_coords = np.radians(dt_subway[['위도', '경도']].values)
bus_coords = np.radians(dt_bus[['Y좌표', 'X좌표']].values)

# BallTree 생성 (Haversine 거리 사용)
sub_tree = BallTree(sub_coords, metric='haversine')
bus_tree = BallTree(bus_coords, metric='haversine')

In [14]:
# 검색 반경 설정 (500m)
radius = 500 / 6371000  # 지구의 평균 반지름으로 나눠서 라디안으로 변환

# 메인 데이터프레임의 좌표 배열 생성
main_coords = np.radians(concat[['좌표Y', '좌표X']].values)

# 각 위치에 대해 반경 내의 점 검색
sub_indices = sub_tree.query_radius(main_coords, r=radius)
bus_indices = bus_tree.query_radius(main_coords, r=radius)

In [15]:
# 주변에 서브 데이터프레임의 위치가 있는지 여부를 저장할 컬럼 생성
concat['지하철여부'] = [len(ind) > 0 for ind in sub_indices]

# 주변에 있는 서브 데이터프레임의 위치 개수 저장
concat['버스정류장개수'] = [len(ind) for ind in bus_indices]

In [16]:
print(concat.head())

             시군구     번지     본번   부번    아파트명  전용면적(㎡)    계약년월  계약일  층  건축년도  \
0  서울특별시 강남구 개포동  658-1  658.0  1.0  개포6차우성    79.97  201712    8  3  1987   
1  서울특별시 강남구 개포동  658-1  658.0  1.0  개포6차우성    79.97  201712   22  4  1987   
2  서울특별시 강남구 개포동  658-1  658.0  1.0  개포6차우성    54.98  201712   28  5  1987   
3  서울특별시 강남구 개포동  658-1  658.0  1.0  개포6차우성    79.97  201801    3  4  1987   
4  서울특별시 강남구 개포동  658-1  658.0  1.0  개포6차우성    79.97  201801    8  2  1987   

   ...                  단지승인일  사용허가여부 관리비 업로드        좌표X        좌표Y  \
0  ...  2022-11-17 13:00:29.0       Y       N  127.05721  37.476763   
1  ...  2022-11-17 13:00:29.0       Y       N  127.05721  37.476763   
2  ...  2022-11-17 13:00:29.0       Y       N  127.05721  37.476763   
3  ...  2022-11-17 13:00:29.0       Y       N  127.05721  37.476763   
4  ...  2022-11-17 13:00:29.0       Y       N  127.05721  37.476763   

                   단지신청일    target is_test  지하철여부 버스정류장개수  
0  2022-11-17 10:19:06.0  124000.0       0  

In [17]:
concat[['도로명', '좌표Y', '좌표X', '지하철여부', '버스정류장개수']].to_csv('balltree_haversion.csv', index=False)